In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from xgboost import XGBClassifier  # Added XGBoost
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
import tensorflow as tf
import time

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load the data
def load_data(file_path):
    df = pd.read_csv(file_path)
    print(f"Loaded data with shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    return df

# Preprocess the text data
def preprocess_text(text):
    if isinstance(text, str):
        # Remove HTML tags
        text = re.sub(r'<.*?>', '', text)
        # Remove URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        # Remove special characters and numbers
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)
        # Convert to lowercase
        text = text.lower()
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ""

# Apply preprocessing to the content column
def preprocess_data(df):
    df['processed_content'] = df['content'].apply(preprocess_text)
    return df

# Vectorize the text data using TF-IDF
def vectorize_text(texts, max_features=5000):
    vectorizer = TfidfVectorizer(max_features=max_features)
    X = vectorizer.fit_transform(texts)
    return X, vectorizer

# Apply KMeans clustering to identify positive, negative, and neutral clusters
def apply_clustering(X, n_clusters=3):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(X)
    return clusters

# Map clusters to sentiment labels (this would typically require manual inspection)
def map_clusters_to_sentiments(df, clusters):
    # This is a placeholder - in a real scenario, you would manually inspect 
    # cluster centers and assign sentiment labels based on that analysis
    cluster_sentiments = {
        0: 'positive',
        1: 'neutral',
        2: 'negative'
    }
    
    df['cluster'] = clusters
    df['sentiment'] = df['cluster'].map(cluster_sentiments)
    
    # Convert to numerical labels for ML algorithms
    sentiment_map = {'positive': 2, 'neutral': 1, 'negative': 0}
    df['sentiment_label'] = df['sentiment'].map(sentiment_map)
    
    return df

# Evaluate traditional ML models
def evaluate_ml_models(X_train, X_test, y_train, y_test):
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'Naive Bayes': MultinomialNB(),
        'SVM': SVC(kernel='linear', random_state=42),
        'XGBoost': XGBClassifier(n_estimators=100, random_state=42),  # Added XGBoost
        'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=42)  # Added AdaBoost
    }
    
    results = {}
    
    for name, model in models.items():
        start_time = time.time()
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        training_time = time.time() - start_time
        
        results[name] = {
            'accuracy': accuracy,
            'training_time': training_time,
            'predictions': predictions,
            'model': model
        }
        
        print(f"{name} Accuracy: {accuracy:.4f}")
        print(f"{name} Training Time: {training_time:.2f} seconds")
        print(classification_report(y_test, predictions))
        print('-' * 50)
    
    return results

# Build and evaluate LSTM model
def build_lstm_model(texts_train, texts_test, y_train, y_test, max_words=10000, max_len=200):
    # Tokenize the text
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(texts_train)
    
    # Convert text to sequences
    sequences_train = tokenizer.texts_to_sequences(texts_train)
    sequences_test = tokenizer.texts_to_sequences(texts_test)
    
    # Pad sequences
    X_train_pad = pad_sequences(sequences_train, maxlen=max_len)
    X_test_pad = pad_sequences(sequences_test, maxlen=max_len)
    
    # Build the model
    model = Sequential()
    model.add(Embedding(max_words, 128, input_length=max_len))
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(Bidirectional(LSTM(32)))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    # Train the model
    start_time = time.time()
    history = model.fit(
        X_train_pad, y_train,
        validation_data=(X_test_pad,y_test),
        epochs=50,
        batch_size=32,
        verbose=1
    )
    training_time = time.time() - start_time
    
    # Evaluate the model
    loss, accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
    predictions = model.predict(X_test_pad)
    predicted_classes = np.argmax(predictions, axis=1)
    
    print(f"LSTM Accuracy: {accuracy:.4f}")
    print(f"LSTM Training Time: {training_time:.2f} seconds")
    print(classification_report(y_test, predicted_classes))
    
    return {
        'accuracy': accuracy,
        'training_time': training_time,
        'predictions': predicted_classes,
        'history': history,
        'model': model
    }

# Plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Negative', 'Neutral', 'Positive'],
                yticklabels=['Negative', 'Neutral', 'Positive'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix - {title}')
    plt.tight_layout()
    plt.savefig(f'confusion_matrix_{title.replace(" ", "_").lower()}.png')
    plt.close()

# Plot accuracy comparison
def plot_accuracy_comparison(results, lstm_result):
    models = list(results.keys()) + ['LSTM']
    accuracies = [results[model]['accuracy'] for model in results] + [lstm_result['accuracy']]
    
    plt.figure(figsize=(14, 6))  # Increased width to accommodate more models
    bars = plt.bar(models, accuracies, color=['blue', 'green', 'red', 'purple', 'cyan', 'brown', 'orange'])
    
    # Add accuracy values on top of the bars
    for bar, acc in zip(bars, accuracies):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                 f'{acc:.4f}', ha='center', va='bottom')
    
    plt.xlabel('Models')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Comparison Across Models')
    plt.ylim(0, 1.1)
    plt.tight_layout()
    plt.savefig('accuracy_comparison.png')
    plt.close()

# Plot training time comparison
def plot_training_time_comparison(results, lstm_result):
    models = list(results.keys()) + ['LSTM']
    times = [results[model]['training_time'] for model in results] + [lstm_result['training_time']]
    
    plt.figure(figsize=(14, 6))  # Increased width to accommodate more models
    bars = plt.bar(models, times, color=['blue', 'green', 'red', 'purple', 'cyan', 'brown', 'orange'])
    
    # Add time values on top of the bars
    for bar, time_val in zip(bars, times):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                 f'{time_val:.2f}s', ha='center', va='bottom')
    
    plt.xlabel('Models')
    plt.ylabel('Training Time (seconds)')
    plt.title('Training Time Comparison Across Models')
    plt.tight_layout()
    plt.savefig('training_time_comparison.png')
    plt.close()

# Plot LSTM training history
def plot_lstm_history(history):
    plt.figure(figsize=(12, 5))
    
    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train')
    plt.plot(history.history['val_accuracy'], label='Validation')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    
    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('lstm_training_history.png')
    plt.close()

# Plot the distribution of clusters/sentiments
def plot_sentiment_distribution(df):
    plt.figure(figsize=(10, 6))
    sns.countplot(x='sentiment', data=df, palette='viridis')
    plt.title('Distribution of Sentiments')
    plt.xlabel('Sentiment')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig('sentiment_distribution.png')
    plt.close()

# Main execution flow
def main(file_path):
    # Load and preprocess data
    df = load_data(file_path)
    df = preprocess_data(df)
    
    # Vectorize the text
    X, vectorizer = vectorize_text(df['processed_content'])
    
    # Apply clustering
    clusters = apply_clustering(X)
    df = map_clusters_to_sentiments(df, clusters)
    
    # Plot sentiment distribution
    plot_sentiment_distribution(df)
    
    # Prepare data for ML models
    X_train, X_test, y_train, y_test = train_test_split(
        X, df['sentiment_label'], test_size=0.2, random_state=42
    )
    
    texts_train, texts_test = train_test_split(
        df['processed_content'], test_size=0.2, random_state=42
    )
    
    # Evaluate ML models
    ml_results = evaluate_ml_models(X_train, X_test, y_train, y_test)
    
    # For each ML model, plot confusion matrix
    for name, result in ml_results.items():
        plot_confusion_matrix(y_test, result['predictions'], name)
    
    # Evaluate LSTM model
    lstm_result = build_lstm_model(texts_train, texts_test, y_train, y_test)
    
    # Plot LSTM confusion matrix
    plot_confusion_matrix(y_test, lstm_result['predictions'], 'LSTM')
    
    # Plot LSTM training history
    plot_lstm_history(lstm_result['history'])
    
    # Plot accuracy comparison
    plot_accuracy_comparison(ml_results, lstm_result)
    
    # Plot training time comparison
    plot_training_time_comparison(ml_results, lstm_result)
    
    return df, ml_results, lstm_result

if __name__ == "__main__":
    # Replace with your actual file path
    file_path = "combined_articles_3316_indiatoday_indiatv_timesofindia_zeenews.csv"
    df, ml_results, lstm_result = main(file_path)
    
    print("\nAnalysis completed successfully!")
    print("\nFinal Accuracy Summary:")
    for model, result in ml_results.items():
        print(f"{model}: {result['accuracy']:.4f}")
    print(f"LSTM: {lstm_result['accuracy']:.4f}")

Loaded data with shape: (3316, 6)
Columns: ['serial_number', 'website_name', 'url', 'title', 'date', 'content']


C:\Users\neeraj\AppData\Local\Temp\ipykernel_26836\4148729949.py:259: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x='sentiment', data=df, palette='viridis')


Logistic Regression Accuracy: 0.9623
Logistic Regression Training Time: 0.42 seconds
              precision    recall  f1-score   support

           0       0.99      0.93      0.96       212
           1       0.95      1.00      0.97       384
           2       1.00      0.85      0.92        68

    accuracy                           0.96       664
   macro avg       0.98      0.93      0.95       664
weighted avg       0.96      0.96      0.96       664

--------------------------------------------------
Random Forest Accuracy: 0.9533
Random Forest Training Time: 2.91 seconds
              precision    recall  f1-score   support

           0       0.98      0.92      0.95       212
           1       0.94      0.99      0.96       384
           2       0.95      0.88      0.92        68

    accuracy                           0.95       664
   macro avg       0.96      0.93      0.94       664
weighted avg       0.95      0.95      0.95       664

-----------------------------

D:\Documents\Zoom\anconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Accuracy: 0.9277
AdaBoost Training Time: 9.01 seconds
              precision    recall  f1-score   support

           0       0.93      0.86      0.90       212
           1       0.92      0.97      0.94       384
           2       0.95      0.91      0.93        68

    accuracy                           0.93       664
   macro avg       0.94      0.91      0.92       664
weighted avg       0.93      0.93      0.93       664

--------------------------------------------------


D:\Documents\Zoom\anconda\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 28s 191ms/step - accuracy: 0.6102 - loss: 0.8985 - val_accuracy: 0.8117 - val_loss: 0.5212
Epoch 2/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 12s 145ms/step - accuracy: 0.8345 - loss: 0.4633 - val_accuracy: 0.8027 - val_loss: 0.4857
Epoch 3/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 13s 153ms/step - accuracy: 0.9022 - loss: 0.3003 - val_accuracy: 0.8645 - val_loss: 0.4080
Epoch 4/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 12s 150ms/step - accuracy: 0.9557 - loss: 0.1575 - val_accuracy: 0.8630 - val_loss: 0.4376
Epoch 5/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 13s 154ms/step - accuracy: 0.9733 - loss: 0.0859 - val_accuracy: 0.8554 - val_loss: 0.5026
Epoch 6/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 13s 162ms/step - accuracy: 0.9802 - loss: 0.0708 - val_accuracy: 0.8645 - val_loss: 0.5850
Epoch 7/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 13s 157ms/step - accuracy: 0.9794 - loss: 0.0699 - val_accuracy: 0.8509 - val_loss: 0.6048
Epoch 8/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 13s 156ms/step - accuracy: 0.9889 - loss: 0.0374 - val_accu

In [2]:
import pandas as pd
import numpy as np
import re
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
import tensorflow as tf

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load the data
def load_data(file_path):
    df = pd.read_csv(file_path)
    print(f"Loaded data with shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    return df

# Preprocess the text data
def preprocess_text(text):
    if isinstance(text, str):
        # Remove HTML tags
        text = re.sub(r'<.*?>', '', text)
        # Remove URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        # Remove special characters and numbers
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)
        # Convert to lowercase
        text = text.lower()
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ""

# Apply preprocessing to the content column
def preprocess_data(df):
    df['processed_content'] = df['content'].apply(preprocess_text)
    return df

# Vectorize the text data using TF-IDF
def vectorize_text(texts, max_features=5000):
    vectorizer = TfidfVectorizer(max_features=max_features)
    X = vectorizer.fit_transform(texts)
    return X, vectorizer

# Apply KMeans clustering to identify positive, negative, and neutral clusters
def apply_clustering(X, n_clusters=3):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(X)
    return clusters

# Map clusters to sentiment labels (this would typically require manual inspection)
def map_clusters_to_sentiments(df, clusters):
    # This is a placeholder - in a real scenario, you would manually inspect 
    # cluster centers and assign sentiment labels based on that analysis
    cluster_sentiments = {
        0: 'positive',
        1: 'neutral',
        2: 'negative'
    }
    
    df['cluster'] = clusters
    df['sentiment'] = df['cluster'].map(cluster_sentiments)
    
    # Convert to numerical labels for ML algorithms
    sentiment_map = {'positive': 2, 'neutral': 1, 'negative': 0}
    df['sentiment_label'] = df['sentiment'].map(sentiment_map)
    
    return df

# Calculate regression metrics for classification problems
def calculate_regression_metrics(y_true, y_pred):
    # Convert predictions to same type as true values
    y_pred = np.array(y_pred, dtype=np.float64)
    y_true = np.array(y_true, dtype=np.float64)
    
    # Calculate metrics
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    return {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2
    }

# Evaluate traditional ML models
def evaluate_ml_models(X_train, X_test, y_train, y_test):
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'Naive Bayes': MultinomialNB(),
        'SVM': SVC(kernel='linear', random_state=42),
        'XGBoost': XGBClassifier(n_estimators=100, random_state=42),
        'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=42)
    }
    
    results = {}
    
    for name, model in models.items():
        start_time = time.time()
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        training_time = time.time() - start_time
        
        # Calculate regression metrics
        reg_metrics = calculate_regression_metrics(y_test, predictions)
        
        results[name] = {
            'accuracy': accuracy,
            'training_time': training_time,
            'predictions': predictions,
            'model': model,
            'mse': reg_metrics['MSE'],
            'rmse': reg_metrics['RMSE'],
            'mae': reg_metrics['MAE'],
            'r2': reg_metrics['R2']
        }
        
        print(f"{name} Metrics:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Training Time: {training_time:.2f} seconds")
        print(f"MSE: {reg_metrics['MSE']:.4f}")
        print(f"RMSE: {reg_metrics['RMSE']:.4f}")
        print(f"MAE: {reg_metrics['MAE']:.4f}")
        print(f"R²: {reg_metrics['R2']:.4f}")
        print(classification_report(y_test, predictions))
        
        # Print confusion matrix in text format
        cm = confusion_matrix(y_test, predictions)
        print("Confusion Matrix:")
        print(cm)
        print('-' * 50)
    
    return results

# Build and evaluate LSTM model
def build_lstm_model(texts_train, texts_test, y_train, y_test, max_words=10000, max_len=200):
    # Tokenize the text
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(texts_train)
    
    # Convert text to sequences
    sequences_train = tokenizer.texts_to_sequences(texts_train)
    sequences_test = tokenizer.texts_to_sequences(texts_test)
    
    # Pad sequences
    X_train_pad = pad_sequences(sequences_train, maxlen=max_len)
    X_test_pad = pad_sequences(sequences_test, maxlen=max_len)
    
    # Build the model
    model = Sequential()
    model.add(Embedding(max_words, 128, input_length=max_len))
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(Bidirectional(LSTM(32)))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    # Train the model
    start_time = time.time()
    history = model.fit(
        X_train_pad, y_train,
        validation_data=(X_test_pad,y_test),
        epochs=10,  # Reduced epochs for faster execution
        batch_size=32,
        verbose=1
    )
    training_time = time.time() - start_time
    
    # Evaluate the model
    loss, accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
    predictions = model.predict(X_test_pad)
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Calculate regression metrics
    reg_metrics = calculate_regression_metrics(y_test, predicted_classes)
    
    print(f"LSTM Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Training Time: {training_time:.2f} seconds")
    print(f"MSE: {reg_metrics['MSE']:.4f}")
    print(f"RMSE: {reg_metrics['RMSE']:.4f}")
    print(f"MAE: {reg_metrics['MAE']:.4f}")
    print(f"R²: {reg_metrics['R2']:.4f}")
    print(classification_report(y_test, predicted_classes))
    
    # Print confusion matrix in text format
    cm = confusion_matrix(y_test, predicted_classes)
    print("Confusion Matrix:")
    print(cm)
    print('-' * 50)
    
    return {
        'accuracy': accuracy,
        'training_time': training_time,
        'predictions': predicted_classes,
        'history': history,
        'model': model,
        'mse': reg_metrics['MSE'],
        'rmse': reg_metrics['RMSE'],
        'mae': reg_metrics['MAE'],
        'r2': reg_metrics['R2']
    }

# Main execution flow
def main(file_path):
    # Load and preprocess data
    df = load_data(file_path)
    df = preprocess_data(df)
    
    # Vectorize the text
    X, vectorizer = vectorize_text(df['processed_content'])
    
    # Apply clustering
    clusters = apply_clustering(X)
    df = map_clusters_to_sentiments(df, clusters)
    
    # Print sentiment distribution
    sentiment_counts = df['sentiment'].value_counts()
    print("\nSentiment Distribution:")
    print(sentiment_counts)
    print(f"Percentage: {sentiment_counts / len(df) * 100}")
    print('-' * 50)
    
    # Prepare data for ML models
    X_train, X_test, y_train, y_test = train_test_split(
        X, df['sentiment_label'], test_size=0.2, random_state=42
    )
    
    texts_train, texts_test = train_test_split(
        df['processed_content'], test_size=0.2, random_state=42
    )
    
    # Evaluate ML models
    ml_results = evaluate_ml_models(X_train, X_test, y_train, y_test)
    
    # Evaluate LSTM model
    lstm_result = build_lstm_model(texts_train, texts_test, y_train, y_test)
    
    return df, ml_results, lstm_result

if __name__ == "__main__":
    # Replace with your actual file path
    file_path = "combined_articles_3316_indiatoday_indiatv_timesofindia_zeenews.csv"
    df, ml_results, lstm_result = main(file_path)
    
    print("\nAnalysis completed successfully!")
    
    print("\nFinal Performance Metrics Summary:")
    metrics_df = pd.DataFrame({
        'Model': list(ml_results.keys()) + ['LSTM'],
        'Accuracy': [ml_results[model]['accuracy'] for model in ml_results] + [lstm_result['accuracy']],
        'MSE': [ml_results[model]['mse'] for model in ml_results] + [lstm_result['mse']],
        'RMSE': [ml_results[model]['rmse'] for model in ml_results] + [lstm_result['rmse']],
        'MAE': [ml_results[model]['mae'] for model in ml_results] + [lstm_result['mae']],
        'R²': [ml_results[model]['r2'] for model in ml_results] + [lstm_result['r2']],
        'Training Time (s)': [ml_results[model]['training_time'] for model in ml_results] + [lstm_result['training_time']]
    })
    
    print(metrics_df.to_string(index=False))
    
    # Save metrics to CSV
    metrics_df.to_csv('model_performance_metrics.csv', index=False)
    
    # Print the best model for each metric
    print("\nBest Models by Metric:")
    print(f"Best Accuracy: {metrics_df.loc[metrics_df['Accuracy'].idxmax()]['Model']} ({metrics_df['Accuracy'].max():.4f})")
    print(f"Lowest MSE: {metrics_df.loc[metrics_df['MSE'].idxmin()]['Model']} ({metrics_df['MSE'].min():.4f})")
    print(f"Lowest RMSE: {metrics_df.loc[metrics_df['RMSE'].idxmin()]['Model']} ({metrics_df['RMSE'].min():.4f})")
    print(f"Lowest MAE: {metrics_df.loc[metrics_df['MAE']0.idxmin()]['Model']} ({metrics_df['MAE'].min():.4f})")
    print(f"Best R²: {metrics_df.loc[metrics_df['R²'].idxmax()]['Model']} ({metrics_df['R²'].max():.4f})")
    print(f"Fastest Training: {metrics_df.loc[metrics_df['Training Time (s)'].idxmin()]['Model']} ({metrics_df['Training Time (s)'].min():.2f}s)")

Loaded data with shape: (3316, 6)
Columns: ['serial_number', 'website_name', 'url', 'title', 'date', 'content']

Sentiment Distribution:
sentiment
neutral     1991
negative     970
positive     355
Name: count, dtype: int64
Percentage: sentiment
neutral     60.042220
negative    29.252111
positive    10.705669
Name: count, dtype: float64
--------------------------------------------------
Logistic Regression Metrics:
Accuracy: 0.9623
Training Time: 0.53 seconds
MSE: 0.0467
RMSE: 0.2161
MAE: 0.0407
R²: 0.8754
              precision    recall  f1-score   support

           0       0.99      0.93      0.96       212
           1       0.95      1.00      0.97       384
           2       1.00      0.85      0.92        68

    accuracy                           0.96       664
   macro avg       0.98      0.93      0.95       664
weighted avg       0.96      0.96      0.96       664

Confusion Matrix:
[[198  14   0]
 [  1 383   0]
 [  2   8  58]]
------------------------------------------

D:\Documents\Zoom\anconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Metrics:
Accuracy: 0.9277
Training Time: 11.69 seconds
MSE: 0.0858
RMSE: 0.2930
MAE: 0.0768
R²: 0.7709
              precision    recall  f1-score   support

           0       0.93      0.86      0.90       212
           1       0.92      0.97      0.94       384
           2       0.95      0.91      0.93        68

    accuracy                           0.93       664
   macro avg       0.94      0.91      0.92       664
weighted avg       0.93      0.93      0.93       664

Confusion Matrix:
[[183  27   2]
 [ 12 371   1]
 [  1   5  62]]
--------------------------------------------------
Epoch 1/10


D:\Documents\Zoom\anconda\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


83/83 ━━━━━━━━━━━━━━━━━━━━ 36s 257ms/step - accuracy: 0.6169 - loss: 0.8907 - val_accuracy: 0.8238 - val_loss: 0.4515
Epoch 2/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 20s 238ms/step - accuracy: 0.8692 - loss: 0.3731 - val_accuracy: 0.8660 - val_loss: 0.3676
Epoch 3/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 20s 240ms/step - accuracy: 0.9198 - loss: 0.2416 - val_accuracy: 0.8630 - val_loss: 0.3910
Epoch 4/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 20s 240ms/step - accuracy: 0.9647 - loss: 0.1200 - val_accuracy: 0.8494 - val_loss: 0.5381
Epoch 5/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 21s 257ms/step - accuracy: 0.9593 - loss: 0.1239 - val_accuracy: 0.8614 - val_loss: 0.6013
Epoch 6/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 18s 220ms/step - accuracy: 0.9769 - loss: 0.0846 - val_accuracy: 0.8494 - val_loss: 0.5818
Epoch 7/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 18s 222ms/step - accuracy: 0.9902 - loss: 0.0384 - val_accuracy: 0.8614 - val_loss: 0.6398
Epoch 8/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 19s 224ms/step - accuracy: 0.9943 - loss: 0.0244 - val_accuracy: 0.858